In [2]:
import numpy as np
import core
import pylab as plt
import nibabel as nib
import os
from skimage import filters
import scipy.ndimage as ndi
from scipy.ndimage.morphology import binary_erosion as be
from scipy.ndimage.morphology import binary_fill_holes as bfh
from scipy.ndimage.morphology import binary_dilation as bd
from scipy.ndimage.morphology import binary_opening as bo
from scipy.ndimage.morphology import binary_closing as bc
from scipy.ndimage.morphology import grey_dilation as gd
import time
from skimage.segmentation import find_boundaries as boundaries
import skimage
import SimpleITK as sitk

In [3]:
import nipype.interfaces.fsl as fsl

def extract(thresholded):
    img = nib.Nifti1Image(thresholded, np.eye(4))
    nib.save(img, 'temp.nii.gz')
    img = nib.load('temp.nii.gz')
    mybet = fsl.BET()
    result = mybet.run(in_file='temp.nii.gz', out_file='BET-temp.nii.gz', frac=0.1)
    img = nib.load('BET-temp.nii.gz')
    data = img.get_data()
    return data

In [12]:
def saveMask(array, path, name):
    mask = array.astype(np.float64)
    mask = np.swapaxes(mask, 0, 1)
    img = nib.Nifti1Image(mask, affine)
    savePath = os.path.join(os.path.split(path)[0], '{}.nii.gz'.format(name))
    nib.save(img, savePath)

In [36]:
info = []
distance = []
planes = np.load('/Volumes/SINEADUSB/Scans/planes-params.npy')

In [37]:
paths = ['/Volumes/SINEADUSB/Scans/ScanB/1.3.12.2.1107.5.1.4.51967.30000014110308064564000003547.dcm', 
        '/Volumes/SINEADUSB/DC-Project-Images/original/Sample2/S2_Post-op.nrrd',
        '/Volumes/SINEADUSB/Scans/ScanD/1.3.12.2.1107.5.1.4.95198.30000015040208110913600035431.dcm',
        '/Volumes/SINEADUSB/Scans/ScanE/1.3.12.2.1107.5.1.4.95198.30000015040208110913600035427.dcm',
        '/Volumes/SINEADUSB/Scans/ScanF/1.3.12.2.1107.5.1.4.95198.30000015061008243097400047435.dcm',
        '/Volumes/SINEADUSB/Scans/ScanG/1.3.12.2.1107.5.1.4.95198.30000015051507521609300034665.dcm',
        '/Volumes/SINEADUSB/Scans/ScanH/1.3.12.2.1107.5.1.4.95198.30000015072708211605400036461.dcm',
        '/Volumes/SINEADUSB/Scans/ScanI/1.3.12.2.1107.5.1.4.95198.30000015071109310409300075672.dcm',
        '/Volumes/SINEADUSB/Scans/ScanJ/1.2.392.200036.9116.2.6.1.48.1221600807.1432222434.802585.dcm']

In [288]:
pixelspacing = []

In [289]:
for pathnumber in range(len(paths)):
    path = paths[pathnumber]
    print 'iteration number', pathnumber

    if path.endswith('.nii') or path.endswith('.nii.gz'):
        array, ConstPixelSpacing, affine = core.nifti2np(path)
        bone = core.thresholdnp(array, 1100, 1500)
        softtissue  = core.thresholdnp(array, 0, 80)

    elif path.endswith('.dcm'):
        array, ConstPixelSpacing, origin, path, affine = core.dicom2np(path)
        bone = core.thresholdnp(array, 900, 1500)
        softtissue  = core.thresholdnp(array, 0, 80)

    elif path.endswith('.nrrd'):
        array, ConstPixelSpacing, affine = core.nrrd2np(path)
    #     array=np.swapaxes(array, 1, 2)
        bone = core.thresholdnp(array, 1100, 1500)
        softtissue  = core.thresholdnp(array, 0, 80)
    
    pixelspacing.append(ConstPixelSpacing)

    # skin
    skin_mask = array

    skin_mask = np.multiply(skin_mask, (array>-200).astype(int))
    array1 = np.multiply(skin_mask, (skin_mask>0.0).astype(int))
    array1 = np.multiply(array1, (array1>-200).astype(int))

    # skull
    skull_mask = array1>84.0
    array2 = np.multiply(array1, (array1<84.0).astype(int))

    # orbital
    orbital_mask = skull_mask

    for i in range(5):
        orbital_mask = bd(skull_mask)

    array3 = np.multiply(array2, (orbital_mask==False).astype(int))

    # cerebral ventricle
    array4 = np.multiply(array3, (array3>0.0).astype(int))

    # BET FSL
    array5 = extract(array4)
    brain_mask = np.multiply(bc(bfh(bd(array5))).astype(int), array5)

    # get ventricles
    ventr_mask1 = np.multiply((brain_mask>0).astype(int), (brain_mask<22).astype(int))
    ventr_mask2 = bd(ventr_mask1)
    ventr_mask3 = be(ventr_mask2)
    ventr_mask4 = bd(ventr_mask3)

    import skimage

    ventr_loop = ventr_mask3

    for i in range(4):
        labels = skimage.measure.label(ventr_loop, connectivity=3)
        props = skimage.measure.regionprops(labels)
        v = [p.area for p in props]
        ind = v.index(max(v))
        vcoords = props[ind].coords
        ventr_loop = np.zeros((array.shape))
        for i in range(len(vcoords)):
            a,b,c = vcoords[i]
            ventr_loop[a,b,c]=1
        ventr_loop = bd(be(ventr_loop))
        
    labels = skimage.measure.label(ventr_loop, connectivity=1)
    props = skimage.measure.regionprops(labels)
    v = [p.area for p in props]
    ind = v.index(max(v))
    vcoords = props[ind].coords
    ventr_loop = np.zeros((array.shape))
    for i in range(len(vcoords)):
        a,b,c = vcoords[i]
        ventr_loop[a,b,c]=1
        
    ventr_loop = bd(be(ventr_loop))

    v_mask = bfh(ventr_loop)
    
    saveMask(v_mask, path, 'vmask')

    # get haematoma

    h_mask = (array5>50.0).astype(float)*1000
    array6 = np.multiply(brain_mask, h_mask)
    array7 = be(array6)
    array8 = bc((array7))
    array9 = bfh((array8))
    array10 = bd(bd((array8)))
    array11 = bfh(bd(array10))
    import skimage

    h_loop = array11

    for i in range(4):
        labels = skimage.measure.label(h_loop, connectivity=3)
        props = skimage.measure.regionprops(labels)
        h = [p.area for p in props]
        ind = h.index(max(h))
        hcoords = props[ind].coords
        h_loop = np.zeros((array.shape))
        for i in range(len(hcoords)):
            a,b,c = hcoords[i]
            h_loop[a,b,c]=1
        h_loop = bd(be(h_loop))
        
    labels = skimage.measure.label(h_loop, connectivity=1)
    props = skimage.measure.regionprops(labels)
    h = [p.area for p in props]
    ind = h.index(max(h))
    hcoords = props[ind].coords
    h_loop = np.zeros((array.shape))
    for i in range(len(hcoords)):
        a,b,c = hcoords[i]
        h_loop[a,b,c]=1
    h_loop = bd(be(h_loop))
    
    h_mask = bfh(h_loop)

    saveMask(h_mask, path, 'hmask')

    # volumes analysis
    plane = planes[pathnumber]
    a,b,c,d = plane[1:].astype(np.float64)

    normal = [a,b,c]

    i= 0
    crossShape = array[:,:,i].shape
    mask = np.zeros(array.shape)

    if np.argmax(normal)==1:
        print 0
        for i in range(bone.shape[2]):
            z = i
            maski = np.fromfunction(lambda x,y: y > (d-c*i-a*x)/b, crossShape)
            mask[:,:,i] = maski

    if np.argmax(normal)==0:
        print 1
        for i in range(bone.shape[2]):
            z = i
            maski = np.fromfunction(lambda x,y: x > ((d-z*c-y*b)/a), crossShape)
            mask[:,:,i] = maski

    midplane_split = np.swapaxes(mask, 0, 1)

    v_mask = (v_mask).astype(int)
    h_mask = (h_mask).astype(int)

    left_h = np.multiply(h_mask, midplane_split==0)
    right_h = np.multiply(h_mask, midplane_split==1)
    
    saveMask(left_h, path, 'lefth')
    saveMask(right_h, path, 'righth')

    left_v = np.multiply(v_mask, midplane_split==0)
    right_v = np.multiply(v_mask, midplane_split==1)
    
    saveMask(left_v, path, 'leftv')
    saveMask(right_v, path, 'rightv')

    brain = np.multiply(brain_mask, (h_mask==0).astype(int)) # no haematoma
    brain_mask = np.multiply(brain_mask, (h_mask==0).astype(int)) # no haematoma
    brain_mask = (brain_mask!=0).astype(int)
    
    saveMask(brain, path, 'brain')
    saveMask(brain_mask, path, 'brainmask')

    left_brain = np.multiply(brain, midplane_split==0)
    right_brain = np.multiply(brain, midplane_split==1)
    

    left_brain_mask = np.multiply(brain_mask, midplane_split==0)
    right_brain_mask = np.multiply(brain_mask, midplane_split==1)

    saveMask(left_brain_mask, path, 'leftbrainmask')
    saveMask(right_brain_mask, path, 'rightbrainmask')
    
    # Ventricle Analysis
    brainArea = []
    for i in range(brain_mask.shape[2]):
        brainArea.append(np.count_nonzero(brain_mask[:,:,i]))
    midsliceIndex = np.argmax(brainArea)

    # check that there is ventricle at the midsliceIndex

    if (np.nonzero(v_mask[:,:,midsliceIndex]) == 0) != False:
        print 'using vArea for vcentroid'
        vArea = []
        for i in range(brain_mask.shape[2]):
            vArea.append(np.count_nonzero(v_mask[:,:,i]))
        midsliceIndex = np.argmax(vArea)

    i = midsliceIndex

    nonzeros = np.nonzero(np.nan_to_num(v_mask[:,:,i]))
    w2, w1 = np.mean(nonzeros[0]), np.mean(nonzeros[1])

    left_h_vol = (left_h[left_h!=0]).size*np.prod(ConstPixelSpacing)
    right_h_vol = (right_h[right_h!=0]).size*np.prod(ConstPixelSpacing)
    left_brain_vol = (left_brain[left_brain!=0]).size*np.prod(ConstPixelSpacing)
    right_brain_vol = (right_brain[right_brain!=0]).size*np.prod(ConstPixelSpacing)
    left_ventr_vol = (left_v[left_v!=0]).size*np.prod(ConstPixelSpacing)
    right_ventr_vol = (right_v[right_v!=0]).size*np.prod(ConstPixelSpacing)

    dirname = os.path.split(os.path.dirname(path))[1]
    
    info.append((dirname, 'w1 = {}'.format(w1), 'w2 = {}'.format(w2),
                 'w3 = {}'.format(midsliceIndex),
                 'Volume of haematoma on LHS is {}'.format(left_h_vol),
                 'Volume of haematoma on RHS is {}'.format(right_h_vol),
                 'Volume of brain on LHS is {}'.format(left_brain_vol),
                 'Volume of brain on RHS is {}'.format(right_brain_vol),
                 'Volume of CSF in ventricles on LHS is {}'.format(left_ventr_vol),
                 'Volume of CSF in ventricles on RHS is {}'.format(right_ventr_vol)))
    
    normal = np.linalg.norm(normal)
    vcentroid = np.array([w1, w2, midsliceIndex])
    pointOnPlane = np.array([0, 0, d/c])
    dist = np.dot(normal, vcentroid - pointOnPlane)
    dist = dist*ConstPixelSpacing
    dist = np.linalg.norm(dist)
    distance.append(('dirname distance is', dist))
    
    print info[pathnumber]

iteration number 0
iteration number 1
iteration number 2
iteration number 3
iteration number 4
iteration number 5
iteration number 6
iteration number 7
iteration number 8


In [41]:
info

[('ScanB',
  'w1 = 287.118632928',
  'w2 = 253.708675649',
  'w3 = 66',
  'Volume of haematoma on LHS is 101216.934341',
  'Volume of haematoma on RHS is 11828.987111',
  'Volume of brain on LHS is 736982.079215',
  'Volume of brain on RHS is 774164.849771',
  'Volume of CSF in ventricles on LHS is 19655.0864758',
  'Volume of CSF in ventricles on RHS is 32386.77633'),
 ('ScanB',
  'w1 = 287.118632928',
  'w2 = 253.708675649',
  'w3 = 66',
  'Volume of haematoma on LHS is 101216.934341',
  'Volume of haematoma on RHS is 11828.987111',
  'Volume of brain on LHS is 736982.079215',
  'Volume of brain on RHS is 774164.849771',
  'Volume of CSF in ventricles on LHS is 19655.0864758',
  'Volume of CSF in ventricles on RHS is 32386.77633'),
 ('Sample2',
  'w1 = 220.927403784',
  'w2 = 366.780924186',
  'w3 = 101',
  'Volume of haematoma on LHS is 43892.5965088',
  'Volume of haematoma on RHS is 39246.1122982',
  'Volume of brain on LHS is 679274.483825',
  'Volume of brain on RHS is 848060.47

In [406]:
pixelspacing = np.load('pixelspacings.npy')

In [407]:
pixelspacing[1] = pixelspacing[0]
pixelspacing = pixelspacing[1:]

In [408]:
pixelspacing = np.array(pixelspacing)

In [409]:
coords = np.array([[287.12, 253.71, 66.0],
            [191.02, 320.46, 143.00],
            [283.02, 273.04, 133.00],
            [230.22, 312.95, 171.00],
            [201.67, 284.08, 95.00],
            [198.90, 300.96, 67.00],
            [217.33, 271.70, 148.00],
            [257.76, 358.19, 72.00]])

np.save('coords.npy', coords)

In [410]:
a_s = planes.T[1]
b_s = planes.T[2]
c_s = planes.T[3]
d_s = planes.T[4]

abcd = np.vstack((b_s, a_s, c_s, d_s)).T.astype(np.float64)
abcd[1] = abcd[0]
abcd = abcd[1:]

normals = abcd.T[:3].T

In [411]:
planepoints = []
for i in range(len(coords)):
    y = 100
    z = 100
    a,b,c,d = abcd[i]
    planepoints = np.array([(d-100*c-100*b)/a,y,z])

In [412]:
normals

array([[ 0.05263394,  0.98944927, -0.13498078],
       [-0.13811027,  0.9860496 , -0.09290717],
       [ 0.00415838,  0.99647491, -0.08378816],
       [ 0.24268522,  0.95917059, -0.14524343],
       [ 0.03986113, -0.99887776, -0.02557946],
       [ 0.02699172,  0.99703744, -0.07202635],
       [-0.14140657,  0.98873267, -0.04911089],
       [-0.32279577,  0.93849006, -0.12263484]])

In [413]:
diffs = planepoints - coords

In [414]:
diffs

array([[-610.82776326, -153.71      ,   34.        ],
       [-514.72776326, -220.46      ,  -43.        ],
       [-606.72776326, -173.04      ,  -33.        ],
       [-553.92776326, -212.95      ,  -71.        ],
       [-525.37776326, -184.08      ,    5.        ],
       [-522.60776326, -200.96      ,   33.        ],
       [-541.03776326, -171.7       ,  -48.        ],
       [-581.46776326, -258.19      ,   28.        ]])

In [415]:
diffs = np.multiply(diffs, pixelspacing)
normals = np.multiply(normals, pixelspacing)


In [418]:
dist = []
for i in range(len(diffs)):
    normals[i] = np.divide(normals[i],np.linalg.norm(normals[i]))
    dist.append(np.dot(diffs[i], normals[i]))

In [422]:
info1= info

In [423]:
info = np.load('info.npy')

In [ ]:
#info contains the scan information

In [ ]:
# iteration number 0
# ('ScanB', 'w1 = 296.916473318', 'w2 = 285.568909513', 'Volume of haematoma on LHS is 113045.921452', 'Volume of haematoma on RHS is 0.0', 'Volume of brain on LHS is 1511146.92899', 'Volume of brain on RHS is 0.0', 'Volume of CSF in ventricles on LHS is 52041.8628058', 'Volume of CSF in ventricles on RHS is 0.0')
# iteration number 1
# ('Sample2', 'w1 = 216.252382818', 'w2 = 365.106084345', 'Volume of haematoma on LHS is 53385.9175234', 'Volume of haematoma on RHS is 29752.7912837', 'Volume of brain on LHS is 792523.571614', 'Volume of brain on RHS is 734811.38909', 'Volume of CSF in ventricles on LHS is 57757.5996118', 'Volume of CSF in ventricles on RHS is 102626.519769')
# iteration number 2
# ('ScanD', 'w1 = 194.920262664', 'w2 = 286.572701689', 'Volume of haematoma on LHS is 56344.5232178', 'Volume of haematoma on RHS is 257760.443811', 'Volume of brain on LHS is 240650.449213', 'Volume of brain on RHS is 857546.920807', 'Volume of CSF in ventricles on LHS is 3839.02203979', 'Volume of CSF in ventricles on RHS is 6914.04701538')
# iteration number 3